# Pick and Place with Motion Planning

In exercises 6.1 (door opening) and 7.2 (mobile IK), we used optimization-based motion planning to solve for the inverse kinematics of our robot arm. In exercise 6.2 (RRT), we implemented the famous sampling-based motion planning algorithms RRT and RRT-Connect. Here, we will put everything together to create a trajectory for picking up our initial and placing it on a shelf.


**Learning Objectives:**
1. Gain more practice definine keyframes for a trajectory
2. Implement IK as optimization 
2. Set up utility functions for RRT-Connect
2. Run Multi-Stage RRT-Connect
3. Implement Shortcutting to fine tune RRT path

**What you'll build:** 
- A simulation of the IIWA placing your inital on a shelf.
- A shortcutting algorithm to smooth and optimize your simulation

Let's start by importing our standard Drake functionality and getting meshcat running

In [1]:
import os
import time
from pathlib import Path
from textwrap import dedent

import numpy as np
from pydrake.all import (
    Context,
    DiagramBuilder,
    InverseKinematics,
    MultibodyPlant,
    PiecewisePolynomial,
    RigidTransform,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.exercises.trajectories.rrt_planner.robot import (
    ConfigurationSpace,
    Range,
)
from manipulation.exercises.trajectories.rrt_planner.rrt_planning import (
    RRT,
    TreeNode,
)
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import LoadScenario, MakeHardwareStation

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


## Part 1: Scene Setup

Our first step will be to set up our scene. We have provided most of the scene setup for you, but you must complete the following task to load your initial into the scene.

**YOUR TASK:** First, create your letter asset by filling in your initial in the cell below. We create two letters, one normal and one "big". The normal letter is the one that we will manipulate in simulation. We use the "big" letter while planning our path to give us a larger margin for error. 

In [3]:
outdir = "scenarios"
assets_dir = "assets"
os.makedirs(outdir, exist_ok=True)
os.makedirs(assets_dir, exist_ok=True)

In [4]:
# TODO: Put your initial here
your_initial = "S"

create_sdf_asset_from_letter(
    text=your_initial,
    font_name="DejaVu Sans",
    letter_height_meters=0.12,
    extrusion_depth_meters=0.05,
    mass=0.01,
    output_dir=Path(assets_dir) / f"{your_initial}_model",
    mu_static=1,
    use_bbox_collision_geometry=True,
)

create_sdf_asset_from_letter(
    text=your_initial,
    font_name="DejaVu Sans",
    letter_height_meters=0.15,
    extrusion_depth_meters=0.08,
    mass=0.01,
    output_dir=Path(assets_dir) / f"{your_initial}_big_model",
    mu_static=1,
    use_bbox_collision_geometry=True,
)

PosixPath('assets/S_big_model/S.sdf')

The next cell creates two yaml files that we will use throughout this exercise. They are written to `scenarios/shelves_scenario.yaml` and `scenarios/shelves_scenario_grasp.yaml`. `shelves_scenario.yaml` contains our normal letter model, and we use it for simulation, planning from the initial pose to the approach pose, and planning from the goal pose to the initial pose. `shelves_scenario_grasp.yaml` contains our "big" letter model, and we use it to plan from the approach pose to the goal pose.

**Note:** Later in this exercise, you may need to edit the pose of your letter defined in `tail_normal` and/or `tail_grasp` below.

In [ ]:
def file_uri(path: str) -> str:
    """Return a file:// URI for an absolute or relative path."""
    abs_path = path if os.path.isabs(path) else os.path.abspath(path)
    return f"file://{abs_path}"


letter_small_uri = file_uri(
    os.path.join(assets_dir, f"{your_initial}_model", f"{your_initial}.sdf")
)
letter_big_uri = file_uri(
    os.path.join(assets_dir, f"{your_initial}_big_model", f"{your_initial}.sdf")
)

base = """
directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0

- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

- add_model:
    name: table
    file: package://manipulation/table.sdf
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
        translation: [0.0, 0.0, -0.05]
        rotation: !Rpy { deg: [0, 0, -90] }

- add_model:
    name: stand
    file: package://manipulation/stand.sdf
- add_weld:
    parent: world
    child: stand::stand_body
    X_PC:
        translation: [0.5, 0.0, 0.0]   # 50 cm in front of iiwa base
        rotation: !Rpy { deg: [0, 0, 0] }

# Shelves
- add_model:
    name: shelves
    file: package://manipulation/shelves.sdf
- add_weld:
    parent: world
    child: shelves::shelves_body
    X_PC:
        translation: [0.9, 0, 0.3995]   # was 0.9057, now ~25cm further in +x
        rotation: !Rpy { deg: [0, 0, 180]}

- add_model:
    name: cheez_it
    file: package://manipulation/hydro/003_cracker_box.sdf
- add_weld:
    parent: shelves::shelves_body
    child: cheez_it::base_link_cracker
    X_PC:
        translation: [0.05, -0.18, 0.245]
        rotation: !Rpy { deg: [-90, 0, 0] }

- add_model:
    name: sugar_box
    file: package://manipulation/hydro/004_sugar_box.sdf
- add_weld:
    parent: shelves::shelves_body
    child: sugar_box::base_link_sugar
    X_PC:
        translation: [0.1, 0.15, -0.035]
        rotation: !Rpy { deg: [-90, 0, -90] }

- add_model:
    name: soup_can
    file: package://manipulation/hydro/005_tomato_soup_can.sdf
- add_weld:
    parent: shelves::shelves_body
    child: soup_can::base_link_soup
    X_PC:
        translation: [0.1, -0.17, -0.072]
        rotation: !Rpy { deg: [-90, 0, -90] }

- add_model:
    name: soup_can2
    file: package://manipulation/hydro/005_tomato_soup_can.sdf
- add_weld:
    parent: shelves::shelves_body
    child: soup_can2::base_link_soup
    X_PC:
        translation: [0.1, -0.17, 0.03]
        rotation: !Rpy { deg: [-90, 0, -90] }

- add_model:
    name: gelatin_box
    file: package://manipulation/hydro/009_gelatin_box.sdf
- add_weld:
    parent: shelves::shelves_body
    child: gelatin_box::base_link_gelatin
    X_PC:
        translation: [0.1, 0.02, -0.087]
        rotation: !Rpy { deg: [-90, 0, -90] }

- add_model:
    name: gelatin_box2
    file: package://manipulation/hydro/009_gelatin_box.sdf
- add_weld:
    parent: shelves::shelves_body
    child: gelatin_box2::base_link_gelatin
    X_PC:
        translation: [0.03, -0.01, -0.087]
        rotation: !Rpy { deg: [-90, 0, -90] }

- add_model:
    name: mustard
    file: package://manipulation/hydro/006_mustard_bottle.sdf
- add_weld:
    parent: stand::stand_body
    child: mustard::base_link_mustard
    X_PC:
        translation: [-0.068, 0.1, 0.263]
        rotation: !Rpy { deg: [0, 0, 0] }
"""

tail_normal = f"""
# Letter placed in the scene (free body pose)
- add_model:
    name: {your_initial}_letter
    file: {letter_small_uri}
    default_free_body_pose:
        {your_initial}_body_link:
            translation: [0.52, -0.05, 0.26]
            rotation: !Rpy {{ deg: [0, 0, 180] }}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

tail_grasp = f"""
# Letter attached to the gripper
- add_model:
    name: {your_initial}_letter
    file: {letter_big_uri}
- add_weld:
    parent: wsg::body
    child: {your_initial}_letter::{your_initial}_body_link
    X_PC:
        translation: [0.03, 0.2, 0.09]
        rotation: !Rpy {{ deg: [0, 90, 180]}}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""


shelves_scenario_yaml = dedent(base + tail_normal).lstrip()
shelves_scenario_grasp_yaml = dedent(base + tail_grasp).lstrip()

# Write files if they don't already exist
path_place = os.path.join(outdir, "shelves_scenario.yaml")
path_grasp = os.path.join(outdir, "shelves_scenario_grasp.yaml")


with open(path_place, "w") as f:
    f.write(shelves_scenario_yaml)

with open(path_grasp, "w") as f:
    f.write(shelves_scenario_grasp_yaml)

In this problem, we will be constructing a path planning algorithm for our robot. TODO so, we need the ability to simulate possible paths and detect wheter or not they create a collision with the environment. This becomes more complicated when we want our robot arm to be grasping an object for a portion of the path. 

The complexity arises from the fact that manipulating our object involves introducing desired collisions. Even initially, there are collisions we desire such as those between objects in the environment and the surfaces they are resting on. When we manipulate an object, we want only a specific set of collisions between the gripper to be allowed. Before moving the object, we are okay with it colliding the table, but after we lift it up, we are not.

To address these complexities, we have defined the class below for you. This provides us with the ability to run our path planning algorithm in two different environments: one when we do not have the letter grasped, and one for when we do. This class also contains some useful methods for our planning algorithm, such as `ExistsCollision` which checks if a given robot configuration results in unwanted contact with the environment.

In [6]:
class ManipulationStationSim:
    def __init__(
        self,
        scenario_file: str | None = None,
        q_iiwa: tuple | None = None,
        gripper_setpoint: float = 0.1,
    ) -> None:

        self.scenario = None
        self.station = None
        self.plant = None
        self.scene_graph = None
        self.query_output_port = None
        self.diagram = None

        # contexts
        self.context_diagram = None
        self.context_station = None
        self.context_scene_graph = None
        self.context_plant = None

        # mark initial configuration
        self.q0 = None

        self.okay_collisions = None
        self.gripper_setpoint = gripper_setpoint

        if scenario_file is not None:
            self.choose_sim(scenario_file, q_iiwa, gripper_setpoint)

    def choose_sim(
        self,
        scenario_file: str,
        q_iiwa: tuple | None = None,
        gripper_setpoint: float = 0.1,
    ) -> None:

        self.clear_meshcat()

        self.scenario = LoadScenario(filename=scenario_file)
        builder = DiagramBuilder()
        self.station = builder.AddSystem(
            MakeHardwareStation(self.scenario, meshcat=meshcat)
        )

        self.plant = self.station.GetSubsystemByName("plant")

        self.scene_graph = self.station.GetSubsystemByName("scene_graph")

        # scene graph query output port.
        self.query_output_port = self.scene_graph.GetOutputPort("query")

        self.diagram = builder.Build()

        # contexts
        self.context_diagram = self.diagram.CreateDefaultContext()
        self.context_station = self.diagram.GetSubsystemContext(
            self.station, self.context_diagram
        )
        self.station.GetInputPort("iiwa.position").FixValue(
            self.context_station, np.zeros(7)
        )
        self.station.GetInputPort("wsg.position").FixValue(self.context_station, [0.1])
        self.context_scene_graph = self.station.GetSubsystemContext(
            self.scene_graph, self.context_station
        )
        self.context_plant = self.station.GetMutableSubsystemContext(
            self.plant, self.context_station
        )

        # mark initial configuration
        self.gripper_setpoint = gripper_setpoint
        if q_iiwa is None:
            self.q0 = self.plant.GetPositions(
                self.context_plant, self.plant.GetModelInstanceByName("iiwa")
            )
        else:
            self.q0 = q_iiwa
            self.SetStationConfiguration(q_iiwa, gripper_setpoint)

        self.DrawStation(self.q0, 0.1)
        query_object = self.query_output_port.Eval(self.context_scene_graph)
        self.okay_collisions = len(query_object.ComputePointPairPenetration())

    def clear_meshcat(self) -> None:
        meshcat.Delete()

    def SetStationConfiguration(self, q_iiwa: tuple, gripper_setpoint: float) -> None:
        """
        :param q_iiwa: (7,) tuple, joint angle of robots in radian.
        :param gripper_setpoint: float, gripper opening distance in meters.
        :return:
        """
        self.plant.SetPositions(
            self.context_plant,
            self.plant.GetModelInstanceByName("iiwa"),
            q_iiwa,
        )
        self.plant.SetPositions(
            self.context_plant,
            self.plant.GetModelInstanceByName("wsg"),
            [-gripper_setpoint / 2, gripper_setpoint / 2],
        )

    def DrawStation(self, q_iiwa: tuple, gripper_setpoint: float = 0.1) -> None:
        self.SetStationConfiguration(q_iiwa, gripper_setpoint)
        self.diagram.ForcedPublish(self.context_diagram)

    def ExistsCollision(self, q_iiwa: tuple, gripper_setpoint: float) -> bool:
        """
        Checks for an unwanted collision for a given robot configuration
        (q_iiwa) and gripper setpoint (gripper_setpoint)

        Args:
            q_iiwa: given robot configuration
            gripper_setpoint: gripper width
        Returns:
            bool: True if an unwnted collision exists, False otherwise
        """

        self.SetStationConfiguration(q_iiwa, gripper_setpoint)
        query_object = self.query_output_port.Eval(self.context_scene_graph)
        collision_pairs = query_object.ComputePointPairPenetration()

        return len(collision_pairs) > self.okay_collisions

Run the following block to load your scene and view it in meshcat!

In [7]:
# Don't change these variable names!
scenario_base_file = os.path.join(outdir, "shelves_scenario.yaml")
scenario_grasp_file = os.path.join(outdir, "shelves_scenario_grasp.yaml")

sim = ManipulationStationSim(scenario_base_file)

Once you have everything set up, the scene should look like this (with your initial in place of the "M"):

**Note:** You may need to adjust the pose of your letter in `tail_normal` (3 cells above) so that it rests on the stand as shown in the image below.

![plan_letters_scenario.png](https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/plan_letters_scenario.png)

## Part 2: Defining the Keyframes

Now that we have our scene all set up, we need to define they keyframes we will use to construct the trajectory. Our goal trajectory is to first grasp the letter and then place it on the top shelf, to the right of the cheez-it box.

**YOUR TASK:** Complete the functions below to define:
- The pregrasp pose
- The grasp pose
- The goal pose

You may recognize that we have a smaller set of poses we must define here compared to previous pick and place exercises. This is because we are using motion planning now! We now just need start and end frames for our motion planning algorithms instead of needing to define more keyframes along the way.

The image below shows what our final keyframes should look like. When defining the keyframes, pay attention to the pose of the gripper relative to the object and relative to the world.

**Note:** The pregrasp pose should result in a gripper pose such that, while holding your letter, neither your gripper or your letter are in contact with other objects in the environment (including the stand). This is *critical* for successfully planning our path. For now, you can just guess what this pose should be. Further below, we provide you with a way to check that your pregrasp configuration is okay.

![plan_letters_keyframes.png](https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/plan_letters_keyframes.png)

In [8]:
def design_grasp_pose(X_WO: RigidTransform) -> RigidTransform:
    # TODO: define your grasp pose here
    # You should return the gripper position relative to the world
    R_OG = RotationMatrix(np.array([[0,0,1],[0,-1,0],[1,0,0]]))
    p_OG = np.array([0.0, 0.13, 0.0])
    X_OG = RigidTransform(R_OG, p_OG)
    X_WG = X_WO @ X_OG
    return X_WG



    # p_WG = X_WO.translation() + np.array([0.0, .1, 0.02])
    # R_WG = RotationMatrix.MakeXRotation(np.pi).multiply(
    #     RotationMatrix.MakeZRotation(np.pi).multiply(RotationMatrix.MakeYRotation(np.pi/2))
    # )
    # return RigidTransform(R_WG, p_WG)


def design_pregrasp_pose(
    X_WG: RigidTransform,
) -> RigidTransform:
    # TODO: define your pregrasp pose here
    # You should return the gripper position relative to the world
    p_WG = X_WG.translation() + np.array([0.0, -.12, 0.0])
    return RigidTransform(X_WG.rotation(), p_WG)


def design_goal_poses() -> RigidTransform:
    # TODO: define your goal pose here, this is where we want to release the object to place it on the shelf
    # You should return the gripper position relative to the world
    p_WG = np.array([0.65, 0, 0.7])
    R_WG = RotationMatrix.MakeYRotation(np.pi/2+.5).multiply(
        RotationMatrix.MakeZRotation(np.pi).multiply(RotationMatrix.MakeXRotation(np.pi/2))
    )
    return RigidTransform(R_WG, p_WG)

Once you have defined the keyframes, run this cell to visualize them in the scene.

**NOTE:** Since our keyframes are defining the start and end of our trajectories, they have a major impact on how the path finding algorithm works. You may find that your initial definition of the keyframes does not perform well, even if they look okay in the scene. Expect to iterate on these keyframe definitions!

In [9]:
# Helper function to express mesh poses in terms of COM rather than geometric center for the letter


def get_initial_pose(
    plant: MultibodyPlant, body_name: str, plant_context: Context
) -> RigidTransform:
    body = plant.GetBodyByName(body_name)
    X_WS = plant.EvalBodyPoseInWorld(plant_context, body)
    X_SO = RigidTransform(body.default_spatial_inertia().get_com())
    return X_WS @ X_SO


# Get initial poses of gripper and objects
X_WGinitial = sim.plant.EvalBodyPoseInWorld(
    sim.context_plant, sim.plant.GetBodyByName("body")
)
X_WOinitial = get_initial_pose(
    sim.plant, f"{your_initial}_body_link", sim.context_plant
)

# Get keyframes
X_WGgrasp = design_grasp_pose(X_WOinitial)
X_WGapproach = design_pregrasp_pose(X_WGgrasp)
X_WGgoal = design_goal_poses()

# Add Triads
AddMeshcatTriad(meshcat, "Initial Pose", X_PT=X_WGinitial, opacity=0.5)
AddMeshcatTriad(meshcat, "Pre-Grasp Pose", X_PT=X_WGapproach, opacity=0.5)
AddMeshcatTriad(meshcat, "Pick Pose", X_PT=X_WGgrasp, opacity=0.5)
AddMeshcatTriad(meshcat, "Place Pose", X_PT=X_WGgoal, opacity=0.5)

## Part 3: IK for Joint Configuration at Keyframes

Now that we have the keyframes we will need to construct our trajectory, we need to find the optimal joint configurations at those keyframes. We can do this using Inverse Kinematics, just like we saw in the door opening and mobile IK notebooks!

**YOUR TASK:** Implement the `solve_ik_for_pose` function below, which takes in a target end effector pose `X_WG_target` and solves IK for the optimal joint configuration to achieve the desired pose. You must implement three components:
- A position constraint such that the end effector should match the desired pose to within `pos_tol` translation along each axis
- A rotation constraint such that the rotation is off by no more than `theta_bound`
- A joint centering cost based on the baseline `q_nominal` passed in. Note that our default `q_nominal` values are the initial joint positions of the iiwa.

**Note:** Do not change the default values for `theta_bound` and `pos_tol`. If you do, your implementation will likely fail the tests below.

In [10]:
def solve_ik_for_pose(
    plant: MultibodyPlant,
    X_WG_target: RigidTransform,
    q_nominal: tuple = tuple(
        np.array([0.0, 0.1, 0.0, -1.2, 0.0, 1.6, 0.0])  # the inital joint poisitions
    ),
    theta_bound: float = 0.01 * np.pi,
    pos_tol: float = 0.015,
) -> tuple:
    """
    Solve IK for a single end-effector pose.

    Args:
        plant: A MultibodyPlant with the iiwa + gripper model.
        X_WG_target: Desired gripper pose in world frame.
        q_nominal: Nominal joint angles for joint-centering.
        theta_bound: Orientation tolerance (radians).
        pos_tol: Position tolerance (meters).

    Returns:
        q_solution: 7 element tuple representing the Optimal
        joint configuration. Each element of the tuple is a float.
    """
    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("body")

    ik = InverseKinematics(plant)
    q_vars = ik.q()[:7]
    prog = ik.prog()

    # TODO: Add Orientation constraint
    R_WG_des = X_WG_target.rotation()
    ik.AddOrientationConstraint(
        frameAbar=world_frame,
        R_AbarA=R_WG_des,           # desired world->gripper rotation
        frameBbar=gripper_frame,    # <-- use frameBbar (not frameB)
        R_BbarB=RotationMatrix(),   # identity in the gripper local frame
        theta_bound=theta_bound,
    )
    # TODO: Add Position constraint
    p_WG_des = X_WG_target.translation()
    p_min = p_WG_des - np.array([pos_tol, pos_tol, pos_tol])
    p_max = p_WG_des + np.array([pos_tol, pos_tol, pos_tol])
    ik.AddPositionConstraint(
        frameB=gripper_frame,
        p_BQ=np.zeros(3),
        frameA=world_frame,
        p_AQ_lower=p_min,
        p_AQ_upper=p_max,
    )
    # TODO: Add Joint centering cost
    error = q_vars - q_nominal
    prog.AddQuadraticCost(error.dot(error))

    # Initial guess
    prog.SetInitialGuess(q_vars, q_nominal)

    result = Solve(prog)
    if not result.is_success():
        raise RuntimeError("IK did not succeed")

    return tuple(result.GetSolution(q_vars))

### Check your Implementation
Run the cell below to test your `solve_ik_for_pose` function. Note that this just checks that IK works as expected, not that your poses are correct.

In [11]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.trajectories.test_plan_place_initials import TestIK_initials

Grader.grade_output([TestIK_initials], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 5/5.

Score for Test solve_ik_for_pose is 5/5.


Now that we have our IK setup, we want to get the optimal joint configurations at our pick and goal poses. 

**YOUR TASK:** Call the `solve_ik_for_pose` function in order to get the joint configurations for the initial, grasp and goal poses. Pay close attention to the nominal joint configurations you are passing in. Think about which nominal configurations we want to use for each goal pose. 

In [12]:
q_initial  = solve_ik_for_pose(sim.plant, X_WGinitial)
q_approach = solve_ik_for_pose(sim.plant, X_WGapproach)
q_grasp    = solve_ik_for_pose(sim.plant, X_WGgrasp,    q_nominal=q_approach)
q_goal = solve_ik_for_pose(sim.plant, X_WGgoal, q_nominal=q_grasp)

# try:
#     q_goal_loose = solve_ik_for_pose(
#         plant=sim.plant,
#         X_WG_target=X_WGgoal,
#         q_nominal=q_grasp,
#         theta_bound=0.95*np.pi,
#         pos_tol=0.06
#     )
#     try:
#         q_goal = solve_ik_for_pose(
#             plant=sim.plant,
#             X_WG_target=X_WGgoal,
#             q_nominal=q_goal_loose,
#             theta_bound=0.08*np.pi,
#             pos_tol=0.04
#         )
#     except RuntimeError:
#         q_goal = q_goal_loose
#         print("Using loose goal solution (acceptable for placement).")
# except RuntimeError as e:
#     print("Goal (loose) failed too:", e)

Run the cell below to visualize the iiwa at the optimal grasp configuration. Feel free to change out `q_grasp` for whichever pose you want to visualize for when the letter is not being grasped.

**Note:** If you need to change the initial pose of your letter on the stand, edit the pose defined in `tail_normal` above and recreate the .yaml files.

In [13]:
sim.choose_sim(scenario_base_file, q_iiwa=q_approach)
sim.DrawStation(q_approach, 0.1)

Run the cell below to visualize the iiwa in the pre-grasp configuration while the letter is being grasped (technically, here the letter is welded to the gripper). An image of what it should look like in the `q_approach` configuration is shown below

**YOUR TASK:** 
- Ensure that neither the gripper nor the letter is in contact with any other object in the environment for the `q_approach` configuration. If the gripper or letter is in contact with the environment, you will need to change your `X_WGapproach`
- Ensure that the letter is concentric with the gripper and is barely touching or has a minimal offset from the gripper base. Essentially, how it might look when being grasped. If not, edit the letter pose defined in `tail_grasp` above and recreate the .yaml files.

**Note:** Here, we use a slightly larger model for the letter. This is to account for small variations of the letter pose relative to the gripper frame.

![plan_letters_grasp.png](https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/plan_letters_grasp.png)

In [14]:
sim.choose_sim(scenario_grasp_file, q_iiwa=q_goal)
sim.DrawStation(q_goal, 0.1)

## Part 4: Implementing RRT-Connect
At this point, we have created our initial model, setup our planning environments, defined our keyframe poses, and found the robot configurations corresponding to those poses. Now, we can finally solve for our collision-free path!


### 4.1 RRT Utilities

Implementing RRT from scratch can be very time-consuming. Below, we have provided you the important features you will need to implement the RRT algorithm.

In [15]:
class RRT_Connect_tools:
    def __init__(
        self,
        sim: ManipulationStationSim,
        start: tuple,
        goal: tuple,
    ) -> None:

        self.sim = sim
        self.start = start
        self.goal = goal

        nq = 7
        joint_limits = np.zeros((nq, 2))
        for i in range(nq):
            joint = sim.plant.GetJointByName("iiwa_joint_%i" % (i + 1))
            joint_limits[i, 0] = joint.position_lower_limits()[0]
            joint_limits[i, 1] = joint.position_upper_limits()[0]

        range_list = []
        for joint_limit in joint_limits:
            range_list.append(Range(joint_limit[0], joint_limit[1]))

        def l2_distance(q: tuple):
            sum = 0
            for q_i in q:
                sum += q_i**2
            return np.sqrt(sum)

        max_steps = nq * [np.pi / 180 * 1.5]  # two degrees
        self.cspace = ConfigurationSpace(range_list, l2_distance, max_steps)
        self.rrt_tree_start = RRT(TreeNode(start), self.cspace)
        self.rrt_tree_goal = RRT(TreeNode(goal), self.cspace)

    def find_nearest_node_in_RRT_graph(self, q_sample: tuple) -> TreeNode:
        """Return nearest node to q_sample in a single-tree context (expects self.rrt_tree)."""
        nearest_node = self.rrt_tree.nearest(q_sample)
        return nearest_node

    def sample_node_in_configuration_space(self) -> tuple:
        """Sample a random valid configuration from the c-space."""
        q_sample = self.cspace.sample()
        return q_sample

    def calc_intermediate_qs_wo_collision(
        self, start: tuple, end: tuple
    ) -> list[tuple]:
        """
        Checks if the path from start to end collides with any obstacles.

        Args:
            start: tuple of floats - tuple describing robot's start
                configuration
            end: tuple of floats - tuple describing robot's end configuration

        Returns:
            list of tuples along the path that are not in collision.
        """
        path = self.cspace.path(start, end)
        safe_path = []
        for configuration in path:
            if self.sim.ExistsCollision(np.array(configuration), 0.1):
                return safe_path
            safe_path.append(configuration)
        return safe_path

    def node_reaches_goal(self, q_step: tuple, tol: float = 1e-2) -> bool:
        """Check if q_step is within tol of the goal in c-space distance."""
        return self.cspace.distance(q_step, self.goal) <= tol

    def backup_path_from_node(self, node: TreeNode) -> list[tuple]:
        """Reconstruct path from tree root to the given node (inclusive)."""
        path = [node.value]
        while node.parent is not None:
            node = node.parent
            path.append(node.value)
        path.reverse()
        return path

    def extend_once(self, tree: RRT, q_target: tuple) -> TreeNode | None:
        """Extend tree by one step toward q_target (returns new node or None if blocked)."""
        q_near_node = tree.nearest(q_target)
        edge = self.calc_intermediate_qs_wo_collision(q_near_node.value, q_target)
        if len(edge) <= 1:
            return None
        q_step = edge[1]
        new_node = tree.add_configuration(q_near_node, q_step)

        return new_node

    def connect_greedy(
        self, tree: RRT, q_target: tuple, eps: float = 1e-2
    ) -> tuple[TreeNode | None, bool]:
        """
        Greedily add as many collision-free segments as possible toward q_target.

        Returns:
            (last_node, complete): last_node reached; complete=True if within eps.
        """
        status = True
        near_node = tree.nearest(q_target)
        q_near_node = near_node.value
        path = self.calc_intermediate_qs_wo_collision(q_near_node, q_target)
        if len(path) > 1:
            last_node = near_node
            for j in range(1, len(path)):
                last_node = tree.add_configuration(last_node, path[j])

            return last_node, (self.cspace.distance(last_node.value, q_target) < eps)

        return (None, False)

    @staticmethod
    def concat_paths(path_a: list[tuple], path_b: list[tuple]) -> list[tuple]:
        """Concatenate two paths, de-duplicating the shared joint at the seam."""
        if path_a and path_b and path_a[-1] == path_b[0]:
            return path_a + path_b[1:]
        return path_a + path_b

### 4.2 RRT Connect

With the utilities defined, we can finally plan our robot's trajectory.

**YOUR TASK**: Complete the function `rrt_connect_planning`. You may find it helpful to reference the methods implemented above and review the RRT-Connect psuedocode in the `rrt_planning` notebook. Note that the utilities provided here are slightly different. As such, the code will need to be slightly different as well.

In [16]:
def rrt_connect_planning(
    sim: "ManipulationStationSim",
    q_start: tuple,
    q_goal: tuple,
    max_iterations: int = 2000,
    eps: float = 1e-2,
) -> tuple[list[tuple] | None, int]:
    """
    Plan a path from q_start to q_goal using RRT-Connect.

    Args:
        sim: a ManipulationStationSim object
        q_start: Start configuration for iiw (tuple of 7 floats)
        q_goal: Goal configuration for iiwa (tuple of 7 floats)
        max_iterations: Maximum RRT expansion attempts.
        eps: Step/connection tolerance

    Returns:
        (path, iterations):
            path: List of configurations from q_start to q_goal if found; otherwise None.
                Each configuration is represented as a tuple of 7 floats
            iterations: Number of iterations performed (<= max_iterations).
    """

    tools = RRT_Connect_tools(sim, start=q_start, goal=q_goal)
    T_start = tools.rrt_tree_start
    T_goal = tools.rrt_tree_goal
    grow_from_start = True  # <-- initialize before loop
    for it in range(max_iterations):
        ## TODO: Sample a location in the configuration space
        q_rand = tools.sample_node_in_configuration_space()
        ## TODO: Select which tree to grow and extend it
        if grow_from_start:
            Ta, Tb = T_start, T_goal
        else:
            Ta, Tb = T_goal, T_start

        new_node = tools.extend_once(Ta, q_rand)

        if new_node is None:
            grow_from_start = not grow_from_start
            continue

        ## TODO: Attempt a greedy connection between the trees
        other_last, complete = tools.connect_greedy(Tb, new_node.value, eps=eps)

        # 5) If connected, build full path and return
        if complete and other_last is not None:
            path_a = tools.backup_path_from_node(new_node)     # from Ta root to meet
            path_b = tools.backup_path_from_node(other_last)   # from Tb root to meet

            if grow_from_start:
                # start -> ... -> meet  ||  meet -> ... -> goal
                full_path = tools.concat_paths(path_a, list(reversed(path_b)))
            else:
                # goal  -> ... -> meet  ||  meet -> ... -> start  (flip order)
                path_a, path_b = path_b, path_a
                full_path = tools.concat_paths(path_a, list(reversed(path_b)))
            print(full_path)
            print("returning")
            return full_path, it + 1

        # Alternate tree for next iteration
        grow_from_start = not grow_from_start
    print("returning None failed")
    return None, max_iterations

### 4.3 Plan the Paths
Run the cell below to construct your paths! Note that you may need to run this multiple times to find valid paths. Feel free to try out different numbers for `max_iterations`. This can take ~1 minute to run

In [17]:
sim.choose_sim(scenario_base_file, q_iiwa=q_initial)
path_pick, num_iter = rrt_connect_planning(
    sim, q_initial, q_approach, max_iterations=3000
)
print(f"Pick Iter: {num_iter}")

sim.choose_sim(scenario_grasp_file, q_iiwa=q_approach)
path_place, num_iter = rrt_connect_planning(
    sim, q_approach, q_goal, max_iterations=8000
)
print(f"Place Iter: {num_iter}")

sim.choose_sim(scenario_base_file, q_iiwa=q_goal)
path_reset, num_iter = rrt_connect_planning(sim, q_goal, q_initial)
print(f"Reset Iter: {num_iter}")

[(np.float64(0.0), np.float64(0.1), np.float64(0.0), np.float64(-1.2), np.float64(0.0), np.float64(1.6), np.float64(0.0)), (np.float64(-0.026995241053608253), np.float64(0.12613101694509177), np.float64(0.009096660673088664), np.float64(-1.1957916916609495), np.float64(0.02070864940318753), np.float64(1.6122380508133038), np.float64(-0.017282299502953152)), (np.float64(-0.053990482107216506), np.float64(0.15226203389018353), np.float64(0.01819332134617733), np.float64(-1.191583383321899), np.float64(0.04141729880637506), np.float64(1.6244761016266078), np.float64(-0.034564599005906305)), (np.float64(-0.08098572316082475), np.float64(0.1783930508352753), np.float64(0.027289982019265992), np.float64(-1.1873750749828487), np.float64(0.06212594820956259), np.float64(1.6367141524399116), np.float64(-0.05184689850885946)), (np.float64(-0.10798096421443301), np.float64(0.20452406778036705), np.float64(0.03638664269235466), np.float64(-1.1831667666437982), np.float64(0.08283459761275012), np.f

Run the cells below to see your paths in action! Do you notice any jittery behavior from your robot? If so, this is completely normal and famously referred to as the "RRT Dance". Don't worry about it for now, we'll deal with it later.

In [18]:
def vis_path(scenario_file: str, path: list[tuple]) -> None:
    if path is not None:
        # show path in meshcat
        sim.choose_sim(scenario_file)
        for q in path:
            q = np.array(q)
            sim.DrawStation(
                q,
                0.1,
            )
            if running_as_notebook:
                time.sleep(0.1)

In [19]:
vis_path(scenario_base_file, path_pick)

In [20]:
vis_path(scenario_grasp_file, path_place)

In [21]:
# Note: you may notice the letter "teleport" back to the table when you visualize this path, that is OK and expected
vis_path(scenario_base_file, path_reset)
print(vis_path(scenario_base_file, path_reset))

None


### Check your Implementation
Run the cell below to test your RRT-Connect function for all 3 paths. Variable names must be unaltered from the original file for the tests to pass. This cell may take ~1 minute to run.

In [22]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.trajectories.test_plan_place_initials import (
    TestRRT_Connect_initials,
)

Grader.grade_output([TestRRT_Connect_initials], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 12/12.

Score for test_path_pick (manipulation.exercises.trajectories.test_plan_place_initials.TestRRT_Connect_initials.test_path_pick) is 3/3.
- [(np.float64(0.0), np.float64(0.1), np.float64(0.0), np.float64(-1.2), np.float64(0.0), np.float64(1.6), np.float64(0.0)), (np.float64(-0.017830617683124137), np.float64(0.11782322976478772), np.float64(0.01881949182452228), np.float64(-1.20919750175049), np.float64(0.013008550557225428), np.float64(1.6034164795417545), np.float64(0.026511282460693245)), (np.float64(-0.04458066461956567), np.float64(0.14298353409474818), np.float64(0.028148780897617784), np.float64(-1.205070946216816), np.float64(0.03305894248626184), np.float64(1.6156655585271442), np.float64(0.008883297630881675)), (np.float64(-0.0713307115560072), np.float64(0.16814383842470862), np.float64(0.03747806997071329), np.float64(-1.2009443906831423), np.float64(0.05310933441529825), np.float64(1.6279146375125337), np.float64(-0.008744687198929894)), (np.float64(-0

## Part 5: Putting it all together  
Now it's time to put our plan in action on the robot!

### 5.1: Construct Trajectory

We planned our path for the robot earlier, but in its current form, it won't run on the robot. Run the cell below to take our paths above and create the corresponding trajectory.

In [23]:
# Assumes: np, PiecewisePolynomial, and these variables exist:
# path_pick, q_grasp, q_approach, path_place, path_reset

dt = 0.05
pause_before_action = 0.5  # pause BEFORE close/open
pause_after_action = 0.5  # pause AFTER  close/open
opened, closed = 0.107, 0.0


def _q7(q: tuple) -> np.ndarray:
    """Ensure a 7-DoF joint vector shaped (7,)."""
    return np.asarray(q, float).reshape(
        7,
    )


def _append_path(
    times: list[float], Q: list[np.ndarray], t: float, path: list[np.ndarray]
) -> float:
    """Append a polyline with uniform dt; returns updated time."""
    for q in path:
        if times:
            t += dt
        times.append(t)
        Q.append(_q7(q))
    return t


def _hold(
    times: list[float],
    Q: list[np.ndarray],
    t: float,
    q_hold: np.ndarray,
    duration: float,
) -> float:
    """Hold current pose for 'duration' seconds; returns updated time."""
    if duration <= 0:
        return t
    t += duration
    times.append(t)
    Q.append(_q7(q_hold))
    return t


def build_trajs(
    path_pick: list[np.ndarray],
    q_grasp: np.ndarray,
    q_approach: np.ndarray,
    path_place: list[np.ndarray],
    path_reset: list[np.ndarray],
) -> tuple["PiecewisePolynomial", "PiecewisePolynomial"]:
    """Sequence:
    pick → q_grasp (OPEN) → pause 0.5 → CLOSE (no motion) → pause 0.5 → q_approach →
    place → pause 0.5 → OPEN (no motion) → pause 0.5 → reset
    """
    times: list[float] = []
    Q: list[np.ndarray] = []
    t = 0.0

    # 1) path_pick  (ends at q_approach typically)
    t = _append_path(times, Q, t, path_pick)

    # 2) move to q_grasp (WSG stays OPEN)
    if not np.allclose(Q[-1], q_grasp):
        t += 10 * dt
        times.append(t)
        Q.append(_q7(q_grasp))

    # 3) pause BEFORE CLOSE (no motion)
    t = _hold(times, Q, t, q_grasp, pause_before_action)

    # 4) CLOSE (command change at this instant), then pause AFTER CLOSE (no motion)
    t_close = t
    t = _hold(times, Q, t, q_grasp, pause_after_action)

    # 5) return to q_approach
    if not np.allclose(Q[-1], q_approach):
        t += 10 * dt
        times.append(t)
        Q.append(_q7(q_approach))

    # 6) path_place
    t = _append_path(times, Q, t, path_place)

    # 7) pause BEFORE OPEN (no motion)
    t = _hold(times, Q, t, Q[-1], pause_before_action)

    # 8) OPEN (command change at this instant), then pause AFTER OPEN (no motion)
    t_open = t
    t = _hold(times, Q, t, Q[-1], pause_after_action)

    # 9) path_reset
    t = _append_path(times, Q, t, path_reset)

    # Build Drake trajectories (7×N for iiwa, 1×K for WSG)
    q_samples = np.stack(Q, axis=1)
    traj_q = PiecewisePolynomial.FirstOrderHold(times, q_samples)

    wsg_knots = [times[0], t_close, t_open, times[-1]]
    wsg_vals = [opened, closed, opened, opened]
    traj_wsg = PiecewisePolynomial.ZeroOrderHold(
        wsg_knots, np.asarray(wsg_vals).reshape(1, -1)
    )

    print(f"T={times[-1]:.3f}s")
    return traj_q, traj_wsg


traj_q, traj_wsg = build_trajs(path_pick, q_grasp, q_approach, path_place, path_reset)

T=12.200s


### 5.2 Setup and Run the Simulation
**YOUR TASK**: Add the trajectories to the diagram, connect them to the robot arm and gripper, and run the simulation!

**Note:** Some trajectories will move very close to objects. As such, small errors in execution can lead to small collisions, which you may see as a flashing contact force in the simulation. This is acceptable. However, your robot should avoid any major collisions with the environment.

In [24]:
# TODO: Load the scenario_base_file scenario
scenario = LoadScenario(filename=scenario_base_file)

# TODO: Create a diagram builder
builder = DiagramBuilder()
# TODO: Make a hardware station with the scenario and our meshcat.
station = builder.AddSystem(MakeHardwareStation(scenario, meshcat=meshcat))
# TODO: Add the trajectories to the diagram as TrajectorySources
q_source = builder.AddSystem(TrajectorySource(traj_q))
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg))

# TODO: Connect the trajectorie source output ports to the arm and gripper
# position input ports, which you can get as "iiwa.position" and "wsg.position"
builder.Connect(q_source.get_output_port(0), station.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(0), station.GetInputPort("wsg.position"))

# TODO: Uncomment the following lines:
diagram = builder.Build()
simulation = Simulator(diagram)

if running_as_notebook:
    simulation.set_target_realtime_rate(1.0)

ctx = simulation.get_mutable_context()
diagram.ForcedPublish(ctx)

meshcat.StartRecording()
simulation.Initialize()
simulation.AdvanceTo(
    max(traj_q.end_time(), traj_wsg.end_time()) if running_as_notebook else 0.1
)
meshcat.StopRecording()
meshcat.PublishRecording()

## Part 6: Shortcutting

In the last problem, you may have noticed the famous "RRT Dance". This is a natural byproduct of the random process used for constructing nodes. There are a lot of algorithms used to improve the path generated by RRT. Here, we explore "shortcutting". 

The pseudocode for shortcutting is shown below.

**Shortcutting**
    
    if path is too short:
        return path

    for t = 1 to passes:
        choose indices i < j with j ≥ i + min_separation // if not possible, break
        q_a ← path[i]
        q_b ← path[j]

        edge ← direct_path_wo_collision(q_a, q_b)   // try a direct, collision-free connection

        if edge successfully reaches q_b:
            path ← ReplaceSubpath(path, i, j, edge)   // splice shortcut in

    path ← Clean(path)   // e.g., drop immediate duplicates, minor tidying
    return path


Here we define some helper functions that you may find useful.

In [25]:
def splice_with_shortcut(
    path: list[tuple], i: int, j: int, edge: list[tuple]
) -> list[tuple]:
    """
    Replace the inclusive subpath path[i:j+1] with 'edge' (which connects
    path[i] -> path[j]). Returns a new path list.
    Assumes edge[0] == path[i] and edge[-1] == path[j].
    """
    prefix = path[:i]
    suffix = path[j + 1 :]
    return prefix + edge + suffix


def check_equal(a: tuple, b: tuple, atol: float = 1e-9) -> bool:
    """Robust waypoint equality for tuples/ndarrays."""
    return np.allclose(np.asarray(a), np.asarray(b), atol=atol, rtol=0.0)

**YOUR TASK**: Implement the `shortcut_path` function below.

In [26]:
def shortcut_path(
    sim: ManipulationStationSim,
    path: list[tuple],
    passes: int = 200,
    min_separation: int = 2,
) -> list[tuple]:
    """
    Randomized shortcutting / smoothing.

    Each pass:
      1) Randomly select indices i < j with j >= i + min_separation.
      2) Try a direct connection using calc_intermediate_qs_wo_collision.
      3) If the connection fully reaches path[j], splice it in.

    Args:
        path: list of configurations (tuples), e.g., [q0, q1, ..., qN].
            each configuration is a tuple of 7 floats
        passes: number of random attempts (upper bound).
        min_separation: require j >= i + min_separation to consider a shortcut.
        rng: optional NumPy Generator for reproducibility (np.random.default_rng(seed)).

    Returns:
        (smoothed_path, num_shortcuts)
    """
    if not path or len(path) < 3:
        return path, 0

    tools = RRT_Connect_tools(sim, path[0], path[-1])
    rng = np.random.default_rng()

    current = path

    for i in range(passes):
        # Recompute length in case prior passes shortened the path.
        n = len(current)
        if n < 3:
            break

        # TODO: finish writing the shortcutting funcion
        i = rng.integers(0, n - min_separation)
        j = rng.integers(i + min_separation, n)
        edge = tools.calc_intermediate_qs_wo_collision(current[i], current[j])
        if len(edge) > 1 and check_equal(edge[-1], current[j]):
            current = splice_with_shortcut(current, i, j, edge)
    # Final dedup (defensive)
    cleaned = []
    prev = None
    for q in current:
        if (prev is None) or not check_equal(q, prev):
            cleaned.append(q)
        prev = q

    return cleaned

Finally, let's run the shortcutting algorithm on the paths constructed earlier and see them in action! Can you see the difference that shortcutting makes?

In [27]:
sim.choose_sim(scenario_base_file, q_iiwa=q_initial)
short_path_pick = shortcut_path(sim, path_pick, passes=200, min_separation=2)

sim.choose_sim(scenario_grasp_file, q_iiwa=q_approach)
short_path_place = shortcut_path(sim, path_place, passes=200, min_separation=2)

sim.choose_sim(scenario_base_file, q_iiwa=q_goal)
short_path_reset = shortcut_path(sim, path_reset, passes=200, min_separation=2)

### Check your Implementation
Run the cell below to test your Shortcutting function. For the test cases to work, you should be passing all RRT-Connect tests above, and all variable names in this file should be unaltered. This cell may take ~1 minute to run.

In [28]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.trajectories.test_plan_place_initials import TestShortcut

Grader.grade_output([TestShortcut], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 10/10.

Score for test_shortcut (manipulation.exercises.trajectories.test_plan_place_initials.TestShortcut.test_shortcut) is 10/10.
- Note: this test does not explicitly verify a shorter path length
[(np.float64(-1.0194378748313089), np.float64(1.1082203479090096), np.float64(0.3393545318556511), np.float64(-1.0833369317846284), np.float64(0.7956513212267468), np.float64(2.0944), np.float64(-0.6721817923270299)), (np.float64(-1.0198040672193662), np.float64(1.0817937144406806), np.float64(0.3657969779796824), np.float64(-1.0609815364941904), np.float64(0.7883385094152502), np.float64(2.0909400498727906), np.float64(-0.6956879937987864)), (np.float64(-1.035641330321325), np.float64(1.055407643880984), np.float64(0.36906190767835484), np.float64(-1.0553527882878935), np.float64(0.7833854183639546), np.float64(2.0859264406198106), np.float64(-0.7106926080193757)), (np.float64(-1.009383684009159), np.float64(1.0401214795381806), np.float64(0.37755722199392056), np.float64(-1

## Setup and Run the Simulation
**YOUR TASK**: Copy your code from above and paste it in the cell below. Then, run the simulation! Is there a noticeable difference from before?

In [29]:
# TODO: Load the scenario_base_file scenario
scenario = LoadScenario(filename=scenario_base_file)

# TODO: Create a diagram builder
builder = DiagramBuilder()
# TODO: Make a hardware station with the scenario and our meshcat.
station = builder.AddSystem(MakeHardwareStation(scenario, meshcat=meshcat))

# TODO: Add the trajectories to the diagram as TrajectorySources
q_source   = builder.AddSystem(TrajectorySource(traj_q))     # 7-dof arm
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg))   # 1-d gripper

# TODO: Connect the trajectorie source output ports to the arm and gripper
# position input ports, which you can get as "iiwa.position" and "wsg.position"
builder.Connect(q_source.get_output_port(),
                station.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(),
                station.GetInputPort("wsg.position"))

# TODO: Uncomment the following lines:
diagram = builder.Build()
simulation = Simulator(diagram)

if running_as_notebook:
    simulation.set_target_realtime_rate(1.0)

ctx = simulation.get_mutable_context()
diagram.ForcedPublish(ctx)

meshcat.StartRecording()
simulation.Initialize()
simulation.AdvanceTo(
    max(traj_q.end_time(), traj_wsg.end_time()) if running_as_notebook else 0.1
)
meshcat.StopRecording()
meshcat.PublishRecording()

# Gradescope Verification

Take a screen recording of the shortcutted trajectory and upload it to gradescope **as an mp4**. The file should be (much) smaller than 500MB. In the video, the should smoothly move to the pre-grasp pose, grasp your initial, place it on the shelf, and return to its starting configuration without colliding with other objects in the environment. Upload the robot following the shortcutted path, so there shouldn't be too much jitter as the robot follows the trajectory.

**Note:** Because some trajectories move very close to objects, small execution errors can lead to slight collisions, visible as flashes of a contact force. This is okay. However, there should not be major collisions with very large forces present.